# 课时23 tf.data输入实例

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
sb.set_style('darkgrid')
import matplotlib.pyplot as plt
import tensorflow as tf
print('Tensorflow Version:', tf.__version__)

Tensorflow Version: 2.4.0


## 1. 加载mnist数据集并进行处理

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images / 255
test_images = test_images / 255

In [6]:
dataset_train_images = tf.data.Dataset.from_tensor_slices(train_images)
dataset_train_labels = tf.data.Dataset.from_tensor_slices(train_labels)
# zip函数将image和label对应起来，传入的是一个元祖
dataset_train = tf.data.Dataset.zip((dataset_train_images, dataset_train_labels))
dataset_train

<ZipDataset shapes: ((28, 28), ()), types: (tf.float64, tf.uint8)>

In [7]:
# 也可以直接将test数据集的图片和标签通过from_tensor_slices转换为Dataset，而不用一个一个的转换再zip。
dataset_test = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
dataset_test = dataset_test.batch(batch_size=64)

## 2. 对数据进行打乱

In [8]:
# 取出其中的10000个数据进行shuffle
dataset_train = dataset_train.shuffle(buffer_size=10000).repeat().batch(batch_size=64)

# 3. 建立模型

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=10, activation='softmax')
])
model.summary()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# 每个epoch需要循环的次数：数据总量 / batch_size
steps_per_epochs = train_images.shape[0] // 64
model.fit(dataset_train, epochs=5, steps_per_epoch=steps_per_epochs,
          validation_data=dataset_test, validation_steps=10000//64)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
937/937 [==============================] - 9s 9ms/step - loss: 0.5130 - accuracy: 0.8586 - val_loss: 0.1610 - val_accuracy: 0.9541
Epoch 2/5
937/937 [==============================] - 7s 7ms/step - loss: 0.1463 - accuracy: 0.9580 - val_loss: 0.1146 - val_accuracy: 0.9675
Epoch 3/5
937/937 [==============================] - 6s 6ms/step - loss: 0.0993 - accuracy: 0.9713